# Download Data
This notebook will download the actual IGRA2 data. By default, we download data for those stations in Tornado Alley that are currently collecting data.

## Data-Por
We are obtaining the data-por version of the IGRA2 data because it contains the full history of observations. The more data the better. By default, we download 23 data files totalling about 1.6 GB in size. You do not need to unzip these files as the olieigra reader can process them in their zipped form.

## Parameterization
Update these parameters to suit your needs

- SILVER_STATION_LIST_PATH - Full path to the igra2-station-list.csv file (See README in this folder if you don't have this file)
- DST_PATH - Folder to download the data files into
- QUERY - SQL Statement to select the stations for which to download data

In [24]:
import os
import pandas as pd
import requests
from pandasql import sqldf

SILVER_STATION_LIST_PATH = '/usr/datalake/silver/igra/doc/igra2-station-list.csv'
SRC_PATH = 'https://www.ncei.noaa.gov/data/integrated-global-radiosonde-archive/access/data-por'
DST_PATH = '/usr/datalake/bronze/igra/data-por'

# Select stations in Tornado Alley that are currently collecting data
QUERY = '''SELECT id, state, name, fst_year, nobs
FROM df 
WHERE STATE IN ('CO', 'IA', 'IL', 'KS', 'MN', 'MO', 'ND', 'NE', 'OK', 'SD', 'TX', 'WI')
    AND lst_year = 2025
ORDER BY state, name'''

In [25]:
# Make sure the destination path exists
os.makedirs(DST_PATH, exist_ok=True)

In [26]:
# Load the CSV and data sanity check
df = pd.read_csv(SILVER_STATION_LIST_PATH, index_col=0, dtype={ 'state': str })
df.head()

,latitude,longitude,elevation,state,name,fst_year,lst_year,nobs
id,,,,,,,,
ACM00078861,17.1170,-61.7830,10.0,NaN,COOLIDGE FIELD (UA),1947,1993,13896
AEM00041217,24.4333,54.6500,16.0,NaN,ABU DHABI INTERNATIONAL AIRPOR,1983,2025,41006
AEXUAE05467,25.2500,55.3700,4.0,NaN,SHARJAH,1935,1942,2477
AFM00040911,36.7000,67.2000,378.0,NaN,MAZAR-I-SHARIF,2010,2014,2179
AFM00040913,36.6667,68.9167,433.0,NaN,KUNDUZ,2010,2013,4540


In [27]:
# Perform the query on the data frame
results = sqldf(QUERY)

# Output which stations we matched
results

,id,state,name,fst_year,nobs
0,USM00072476,CO,GRAND JUNCTION/WALKER FIELD; C,1938,70160
1,USM00074455,IA,QUAD CITY; IA.,1935,24618
2,USM00074560,IL,LINCOLN; IL.,1995,22785
3,USM00072451,KS,DODGE CITY/MUN.; KS.,1940,66859
4,USM00072456,KS,TOPEKA/MUN.; KS.,1953,55476
5,USM00072649,MN,CHANHASSEN; MN.,1937,28134
6,USM00072747,MN,INT.FALLS/FALLS INT. MN.,1942,62693
7,USM00072440,MO,SPRINGFIELD/MUN.; MO.,1939,33990
8,USM00072764,ND,BISMARCK/MUN.; ND.,1932,80499
9,USM00072562,NE,NORTH PLATTE/LEE BIRD; NE.,1930,77468


In [28]:
for _, station in results.iterrows():
    filename = f"{station.id}-data.txt.zip"
    url = f"{SRC_PATH}/{filename}"
    local = f"{DST_PATH}/{filename}"

    if os.path.exists(local):
        print(f"File already exists: {local}")
        continue

    r = requests.get(url)
    open(local, 'wb').write(r.content)
    print(f"Downloaded {local}")

Downloaded /usr/datalake/bronze/igra/data-por/USM00072476-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00074455-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00074560-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072451-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072456-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072649-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072747-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072440-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072764-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072562-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072558-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00074646-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072357-data.txt.zip
Downloaded /usr/datalake/bronze/igra/data-por/USM00072659-data.txt.zip
Downlo